## MIGHT data - real

In [5]:
###############################################
# 1) Imports and Basic Setup
###############################################
import time
import pandas as pd
import numpy as np

# This is the current Python package for Yggdrasil Decision Forests.
import ydf

###############################################
# 2) Load the Exact Same Processed Data
###############################################
# In your MIGHT notebook, you saved something like "processed_wise1_data.csv".
# Let's read that in so YDF sees the identical data.

PROCESSED_DATA = "processed_wise1_data.csv"
df = pd.read_csv(PROCESSED_DATA)

# Suppose the label column is named "Cancer Status".
LABEL_COL = "Cancer Status"

print("DataFrame shape:", df.shape)
# print("Columns:", df.columns.tolist())

###############################################
# 3) Verify We Have the Same Classification Task
###############################################
# Ensure that the label distribution matches what you see in MIGHT (e.g., y.value_counts()).

num_rows, num_cols = df.shape
if LABEL_COL not in df.columns:
    raise ValueError(f"Label column {LABEL_COL!r} not found in CSV columns!")

print("Number of samples (rows):", num_rows)
print("Number of columns:", num_cols)
print(f"Label '{LABEL_COL}' distribution:\n{df[LABEL_COL].value_counts()}")

# Optional: Print the first few columns to confirm
print("Feature columns (excluding label):")
feature_cols = [c for c in df.columns if c != LABEL_COL]
print(feature_cols[:10], "..." if len(feature_cols) > 10 else "")

###############################################
# 4) Train a YDF Random Forest
###############################################
# Provide the label name, and optionally set random_seed for reproducibility.

rf_learner = ydf.RandomForestLearner(
    label=LABEL_COL,
    random_seed=42,  # ensures reproducible random sampling
    num_trees=100,   # match MIGHT for fair comparison
    max_depth=10     # or whatever matches your MIGHT hyperparams
)

start_time = time.perf_counter()
rf_model = rf_learner.train(df)
end_time = time.perf_counter()

train_time = end_time - start_time
print(f"\n[INFO] Yggdrasil RF trained in {train_time:.2f} seconds.")

###############################################
# 5) Evaluate / Inspect the Model
###############################################
# Evaluate on the same data (for quick demonstration).
evaluation = rf_model.evaluate(df)
print("\nEvaluation metrics on the training set:")
print(evaluation)

# If you want, you can also get predictions or partial-dependence analysis:
predictions = rf_model.predict(df)
print("\nSample predictions (first 5 rows):")
print(predictions)

###############################################
# 6) Confirm Similarity with MIGHT
###############################################
# For a fair side-by-side timing:
# - MIGHT uses the same "processed_wise1_data.csv".
# - Both have 100 trees, same random seed if possible.
# - Start/stop the timer at the exact training call.
#
# Now the only difference should be the algorithms/impl details themselves.
###############################################

DataFrame shape: (352, 2524)
Number of samples (rows): 352
Number of columns: 2524
Label 'Cancer Status' distribution:
Cancer Status
0    250
1    102
Name: count, dtype: int64
Feature columns (excluding label):
['1:1000001-2000000', '1:3000001-4000000', '1:4000001-5000000', '1:5000001-6000000', '1:6000001-7000000', '1:7000001-8000000', '1:8000001-9000000', '1:9000001-10000000', '1:10000001-11000000', '1:11000001-12000000'] ...
Train model on 352 examples
Model trained in 0:00:00.135285

[INFO] Yggdrasil RF trained in 0.55 seconds.

Evaluation metrics on the training set:
accuracy: 1
confusion matrix:
    label (row) \ prediction (col)
    +-----+-----+-----+
    |     |   0 |   1 |
    +-----+-----+-----+
    |   0 | 250 |   0 |
    +-----+-----+-----+
    |   1 |   0 | 102 |
    +-----+-----+-----+
characteristics:
    name: '1' vs others
    ROC AUC: 1
    PR AUC: 1
    Num thresholds: 58
loss: 0.164877
num examples: 352
num examples (weighted): 352


Sample predictions (first 5 row

In [ ]:
import pandas as pd
import numpy as np
import time

import ydf  # Yggdrasil Decision Forests
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1) Read the same processed CSV as MIGHT
CSV_FILE = "processed_wise1_data.csv"
df = pd.read_csv(CSV_FILE)

LABEL_COL = "Cancer Status"
if LABEL_COL not in df.columns:
    raise ValueError(f"Missing label column {LABEL_COL!r} in CSV.")

# 2) Train/test split (hold-out)
#    We'll separate 30% of the data for testing. Use the same random_state so you can replicate in MIGHT.
X = df.drop(columns=[LABEL_COL])
y = df[LABEL_COL]
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=y)

print("Train set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

# 3) Build a YDF RandomForest with ~the same hyperparams as MIGHT
#    e.g. 100 trees, random_seed=23 or 42, etc.
rf_learner = ydf.RandomForestLearner(
    label=LABEL_COL,
    random_seed=42,
    num_trees=100 #100000
)

# 4) Train on the HOLD-OUT train set
start_time = time.perf_counter()
rf_model = rf_learner.train(train_df)  # Only pass the train portion
end_time = time.perf_counter()
train_time = end_time - start_time
print(f"\nYDF model trained in {train_time:.2f} seconds on {len(train_df)} examples.")

# 5) Evaluate on the test set using YDF's built-in evaluate()
evaluation = rf_model.evaluate(test_df)
print("\n=== YDF Evaluate() on Test Set ===")
print(evaluation)

#   By default, it prints metrics like "accuracy", "confusion matrix", "ROC AUC",
#   etc. in a structured text output.

# 6) (Optional) Compute scikit-learn metrics on the test set
#    The YDF model's `.predict()` returns a DataFrame with probabilities for label=1 by default.
preds = rf_model.predict(test_df)  # shape: (num_test_examples,) containing probabilities
preds_np = preds#.to_numpy().ravel()

# Convert probabilities -> predicted class using 0.5 threshold
pred_labels = (preds_np >= 0.5).astype(int)

test_y = test_df[LABEL_COL].values

acc = accuracy_score(test_y, pred_labels)
cm = confusion_matrix(test_y, pred_labels)
cls_rpt = classification_report(test_y, pred_labels)

print("\n=== Scikit-learn metrics on Test Set ===")
print(f"Accuracy: {acc:.3f}")
print("Confusion matrix:\n", cm)
print("\nClassification Report:\n", cls_rpt)

# 7) Display sample predicted probabilities (first 10), to mimic the style you saw
print("\nSample predicted probabilities (first 10):")
print(preds_np[:10])

Train set shape: (246, 2524)
Test set shape: (106, 2524)
Train model on 246 examples
Model trained in 0:00:00.107743

YDF model trained in 0.17 seconds on 246 examples.

=== YDF Evaluate() on Test Set ===
accuracy: 0.811321
confusion matrix:
    label (row) \ prediction (col)
    +----+----+----+
    |    |  0 |  1 |
    +----+----+----+
    |  0 | 75 |  0 |
    +----+----+----+
    |  1 | 20 | 11 |
    +----+----+----+
characteristics:
    name: '1' vs others
    ROC AUC: 0.943871
    PR AUC: 0.891002
    Num thresholds: 46
loss: 0.426873
num examples: 106
num examples (weighted): 106


=== Scikit-learn metrics on Test Set ===
Accuracy: 0.811
Confusion matrix:
 [[75  0]
 [20 11]]

Classification Report:
               precision    recall  f1-score   support

           0       0.79      1.00      0.88        75
           1       1.00      0.35      0.52        31

    accuracy                           0.81       106
   macro avg       0.89      0.68      0.70       106
weighted avg 

# Synthetic Random Data

## Oblique Splits

In [1]:
def plot_3d_bars(pivot_df, title="3D Bar Plot", zlabel="Runtime (s)"):
    """
    Same 3D bar chart method as in the MIGHT code.
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    n_values = pivot_df.index.to_numpy()
    d_values = pivot_df.columns.to_numpy()

    n_indices = np.arange(len(n_values))
    d_indices = np.arange(len(d_values))

    bar_width = 0.7
    xs = []
    ys = []
    dzs = []

    for i, n_val in enumerate(n_values):
        for j, d_val in enumerate(d_values):
            xs.append(j)
            ys.append(i)
            dzs.append(pivot_df.iloc[i, j])

    xs = np.array(xs)
    ys = np.array(ys)
    zs = np.zeros_like(xs)
    dx = bar_width * np.ones_like(xs)
    dy = bar_width * np.ones_like(xs)
    dz = np.array(dzs)

    ax.bar3d(xs, ys, zs, dx, dy, dz)

    ax.set_title(title)
    ax.set_xlabel("d (features)")
    ax.set_ylabel("n (rows)")
    ax.set_zlabel(zlabel)

    ax.set_xticks(d_indices + bar_width / 2)
    ax.set_xticklabels(d_values)

    ax.set_yticks(n_indices + bar_width / 2)
    ax.set_yticklabels(n_values)

    plt.show()

### set num_attributes to 160

🔬 <font color="purple">This will test Cache performance - at first 160/160 features - dense access. Then 160/320 features - 50% sparse access ... </font>

In [ ]:
##############################################
# YDF Benchmark with Repeats & 3D Plot
##############################################

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

import ydf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def benchmark_ydf(n_vals, d_vals, repeats=7, random_seed=42):
    """
    Benchmarks YDF's training & inference time across multiple (n, d) data sizes,
    repeating each (n,d) `repeats` times to compute mean & std dev.

    Returns a DataFrame with columns:
      [n, d,
       train_time_mean, train_time_std,
       inference_time_mean, inference_time_std,
       accuracy_mean, accuracy_std]
    """
    np.random.seed(random_seed)
    rows = []

    for n in n_vals:
        for d in d_vals:
            train_times = []
            inference_times = []
            accuracies = []

            for _ in range(repeats):
                # 1) Generate random data
                X = np.random.randn(n, d)
                y = np.random.randint(2, size=n)

                # 2) Split
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.3, random_state=random_seed, stratify=y
                )

                # 3) Convert to DF with string column names
                cols = [f"X{i}" for i in range(d)]
                train_df = pd.DataFrame(X_train, columns=cols)
                train_df["label"] = y_train

                test_df = pd.DataFrame(X_test, columns=cols)
                test_df["label"] = y_test

                # 4) YDF RandomForest
                rf_learner = ydf.RandomForestLearner(
                    label="label",
                    random_seed=random_seed,
                    split_axis="SPARSE_OBLIQUE",
                    num_trees=1000,
                    bootstrap_training_dataset=True,
                    bootstrap_size_ratio=1.0,
                    num_threads=96,
                    num_candidate_attributes=160,
                    max_depth=-1,
                    honest=True
                )

                # 5) Training time
                t0 = time.perf_counter()
                model = rf_learner.train(train_df)
                t1 = time.perf_counter()
                train_times.append(t1 - t0)

                # 6) Inference time
                t2 = time.perf_counter()
                preds_df = model.predict(test_df)  # prob of label=1
                t3 = time.perf_counter()
                inference_times.append(t3 - t2)

                # Convert probability -> predicted label
                preds_np = preds_df#.values.ravel()
                pred_labels = (preds_np >= 0.5).astype(int)
                acc = accuracy_score(y_test, pred_labels)
                accuracies.append(acc)

            # Mean/std
            train_time_mean = np.mean(train_times)
            train_time_std  = np.std(train_times)
            inf_time_mean   = np.mean(inference_times)
            inf_time_std    = np.std(inference_times)
            acc_mean        = np.mean(accuracies)
            acc_std         = np.std(accuracies)

            print(f"\n(n={n}, d={d}):")
            print(f"  train_time mean={train_time_mean:.4f}s, std={train_time_std:.4f}")
            print(f"  inference_time mean={inf_time_mean:.4f}s, std={inf_time_std:.4f}")
            print(f"  accuracy mean={acc_mean:.3f}, std={acc_std:.3f}")

            rows.append({
                "n": n,
                "d": d,
                "train_time_mean": train_time_mean,
                "train_time_std": train_time_std,
                "inference_time_mean": inf_time_mean,
                "inference_time_std": inf_time_std,
                "accuracy_mean": acc_mean,
                "accuracy_std": acc_std
            })

    return pd.DataFrame(rows)


if __name__ == "__main__":
    # Example parameter grids
    n_values = [500, 1000, 2000, 4000, 8000]#, 16000, 32000, 64000]
    d_values = [160, 320, 640, 1024, 2048, 4096, 8192]

    df_bench = benchmark_ydf(n_values, d_values, repeats=1, random_seed=42)

    print("\n=== YDF Benchmark Results (Averaged Over Repeats) ===")
    print(df_bench)

    # pivot_table for train_time_mean
    pivot_train = df_bench.pivot(index="n", columns="d", values="train_time_mean")
    pivot_infer = df_bench.pivot(index="n", columns="d", values="inference_time_mean")

    # ---- 1) Heatmap: Training Time
    plt.figure()
    plt.imshow(pivot_train, aspect="auto")
    plt.colorbar()
    plt.title("YDF Training Time Mean (sec)")
    plt.xticks(range(len(d_values)), d_values)
    plt.yticks(range(len(n_values)), n_values)
    plt.xlabel("Features (d)")
    plt.ylabel("Rows (n)")
    plt.show()

    # ---- 2) Heatmap: Inference Time
    plt.figure()
    plt.imshow(pivot_infer, aspect="auto")
    plt.colorbar()
    plt.title("YDF Inference Time Mean (sec)")
    plt.xticks(range(len(d_values)), d_values)
    plt.yticks(range(len(n_values)), n_values)
    plt.xlabel("Features (d)")
    plt.ylabel("Rows (n)")
    plt.show()

    # ---- 3) 3D "Cityscape" for Training Time
    plot_3d_bars(
        pivot_train,
        title="YDF Training Time (3D Bar)",
        zlabel="Train time (s)"
    )

    # ---- 4) 3D "Cityscape" for Inference Time
    plot_3d_bars(
        pivot_infer,
        title="YDF Inference Time (3D Bar)",
        zlabel="Inference time (s)"
    )

Train model on 350 examples
Model trained in 0:00:09.345808

(n=500, d=160):
  train_time mean=9.3544s, std=0.0000
  inference_time mean=0.0237s, std=0.0000
  accuracy mean=0.487, std=0.000
Train model on 350 examples
Model trained in 0:00:13.366411

(n=500, d=320):
  train_time mean=13.3938s, std=0.0000
  inference_time mean=0.0354s, std=0.0000
  accuracy mean=0.547, std=0.000
Train model on 350 examples
Model trained in 0:00:20.933524

(n=500, d=640):
  train_time mean=20.9736s, std=0.0000
  inference_time mean=0.0473s, std=0.0000
  accuracy mean=0.500, std=0.000
Train model on 350 examples
Model trained in 0:00:30.524666

(n=500, d=1024):
  train_time mean=30.5829s, std=0.0000
  inference_time mean=0.1934s, std=0.0000
  accuracy mean=0.560, std=0.000
Train model on 350 examples
Model trained in 0:00:55.414250

(n=500, d=2048):
  train_time mean=55.5162s, std=0.0000
  inference_time mean=0.1008s, std=0.0000
  accuracy mean=0.533, std=0.000
Train model on 350 examples
Model trained in

### Profiling

Profiling in python in useless. Should profile in C

In [6]:
def train_ydf_model(train_df):
    """Trains a YDF model (RandomForest) and returns it."""
    rf_learner = ydf.RandomForestLearner(
        label="label",
        random_seed=42,
        split_axis="SPARSE_OBLIQUE",
        num_trees=1000,
        bootstrap_training_dataset=True,
        bootstrap_size_ratio=1.0,
        num_threads=1, # Limit to 1 thread for Profiling
        num_candidate_attributes=160,
        max_depth=-1,
        honest=True
    )
    return rf_learner.train(train_df)
    

def benchmark_ydf_static_data(n_vals, d_vals, repeats=7, random_seed=42):
    """
    Benchmarks YDF training & inference with pre-generated datasets.
    """
    np.random.seed(random_seed)
    rows = []

    for n in n_vals:
        for d in d_vals:
            # === Pre-generate data ===
            X = np.random.randn(n, d)
            y = np.random.randint(2, size=n)
            cols = [f"X{i}" for i in range(d)]

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.3, random_state=random_seed, stratify=y
            )

            train_df = pd.DataFrame(X_train, columns=cols)
            train_df["label"] = y_train

            test_df = pd.DataFrame(X_test, columns=cols)
            test_df["label"] = y_test

            train_times = []
            inference_times = []
            accuracies = []

            for _ in range(repeats):
                # === Train ===
                t0 = time.perf_counter()
                model = train_ydf_model(train_df)
                # Save the model code to model.h and display it
                with open("ydf_tutorial_model.h", "w") as f:
                  f.write(model.to_cpp(key="ydf_tutorial"))
                print("Saved model to ydf_tutorial_model.h!")
                
                # !cat ydf_tutorial_model.h
                t1 = time.perf_counter()
                train_times.append(t1 - t0)

                # === Predict ===
                t2 = time.perf_counter()
                preds_df = model.predict(test_df)
                t3 = time.perf_counter()
                inference_times.append(t3 - t2)

                pred_labels = (preds_df >= 0.5).astype(int)
                acc = accuracy_score(y_test, pred_labels)
                accuracies.append(acc)

            # === Aggregate ===
            rows.append({
                "n": n,
                "d": d,
                "train_time_mean": np.mean(train_times),
                "train_time_std": np.std(train_times),
                "inference_time_mean": np.mean(inference_times),
                "inference_time_std": np.std(inference_times),
                "accuracy_mean": np.mean(accuracies),
                "accuracy_std": np.std(accuracies),
            })

            print(f"\n(n={n}, d={d}):")
            print(f"  train_time mean={np.mean(train_times):.4f}s, std={np.std(train_times):.4f}")
            print(f"  inference_time mean={np.mean(inference_times):.4f}s, std={np.std(inference_times):.4f}")
            print(f"  accuracy mean={np.mean(accuracies):.3f}, std={np.std(accuracies):.3f}")

    return pd.DataFrame(rows)

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

import ydf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def generate_ydf_data(n, d, seed=42):
    np.random.seed(seed)
    X = np.random.randn(n, d)
    y = np.random.randint(2, size=n)
    cols = [f"X{i}" for i in range(d)]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed, stratify=y
    )

    train_df = pd.DataFrame(X_train, columns=cols)
    train_df["label"] = y_train

    test_df = pd.DataFrame(X_test, columns=cols)
    test_df["label"] = y_test

    return train_df, test_df, y_test


n, d = 4000, 2048
train_df, test_df, y_test = generate_ydf_data(n, d)

In [4]:
test_df.to_csv("ydf_random_test_set.csv")

In [5]:
import cProfile
import pstats
from io import StringIO

pr = cProfile.Profile()
pr.enable()

# Call your benchmark or just train_ydf_model()
train_ydf_model(train_df)

pr.disable()

# Print top functions by cumulative time
s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats("cumulative")
ps.print_stats(20)  # top 20 functions
print(s.getvalue())

Train model on 70 examples
Model trained in 0:00:05.826759
         8861 function calls (8266 primitive calls) in 5.828 seconds

   Ordered by: cumulative time
   List reduced from 369 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    11/10    0.000    0.000    5.824    0.582 /usr/lib/python3.12/asyncio/base_events.py:1910(_run_once)
        6    4.333    0.722    4.335    0.723 {built-in method time.sleep}
    11/10    0.356    0.032    3.487    0.349 /usr/lib/python3.12/selectors.py:451(select)
       10    1.130    0.113    1.130    0.113 {method 'poll' of 'select.epoll' objects}
       12    0.000    0.000    0.027    0.002 /home/ubuntu/ydf-env/lib/python3.12/site-packages/ydf/dataset/dataset.py:106(_add_column)
      2/1    0.000    0.000    0.002    0.002 /home/ubuntu/ydf-env/lib/python3.12/site-packages/ydf/utils/log.py:127(info)
        2    0.000    0.000    0.002    0.001 /home/ubuntu/ydf-env/lib/python3.12/site-package